In [1]:
import numpy as np
import scipy as sp

class SensorData:
    
    def __init__(self, sensorTimestampMs, x1, y1, z1, x2, y2, z2):

        self.sensorTimestampMs = int(sensorTimestampMs)
        
        self.x1 = float(x1)
        self.y1 = float(y1)
        self.z1 = float(z1)
        
        self.x2 = float(x2)
        self.y2 = float(y2)
        self.z2 = float(z2)

        self.mag1 = np.sqrt(self.x1**2 + self.y1**2 + self.z1**2)
        self.mag2 = np.sqrt(self.x2**2 + self.y2**2 + self.z2**2)

In [2]:
# read in data files to construct posture and walking data sets
from os import listdir
import ntpath
import os

good_posture = []
bad_posture = []

walking_segments = []
not_walking_segments = []

# number of samples to accumulate
segment_length = 20

data_dir = "./data/"
filenames = listdir(data_dir)

for filename in filenames:
    samples = []
    
    #open each data file and convert to list of data objects
    file = open(data_dir+filename, "r")
    for data_line in file:
        s_data = data_line.split(",")
        if len(s_data) == 7:
            samples.append(SensorData(*s_data))
    
    # create posture data set
    if "good" in filename:
        good_posture.extend(samples)
    else:
        bad_posture.extend(samples)
                
    #create walking data set
    sample_segments = []
    #break samples list into segments of segment_length
    for i in range(0, len(samples), segment_length):
        sample_segments.append(samples[i:i+segment_length])
    
    if "walking" in filename:
        walking_segments.extend(sample_segments)
    else:
        not_walking_segments.extend(sample_segments)
        
            
print(len(good_posture))
print(len(bad_posture))

print(len(walking_segments))
print(len(not_walking_segments))

1038
1132
33
82


In [9]:
# extract features from single sensor data object
def extract_signal_features(s_data):
    features = []
    
    features.append(s_data.x1)
    features.append(s_data.y1)
    features.append(s_data.z1)
    
    features.append(s_data.x2)
    features.append(s_data.y2)
    features.append(s_data.z2)
    
    #dot product
    #features.append(s_data.x1 * s_data.x2 + s_data.y1 * s_data.y2 + s_data.z1 * s_data.z2)
    #features.append(s_data.x1 - s_data.x2)
    #features.append(s_data.y1 - s_data.y2)
    #features.append(s_data.z1 - s_data.z2)
    
    return features

In [10]:
# train and test posture model
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
import itertools
import time
import random
import numpy as np
import scipy as sp
from scipy import stats, signal

#randomize data sets and 80-20 train-test split them
#https://stackoverflow.com/questions/13423759/percent-list-slicing
random.shuffle(good_posture)
train_good = good_posture[0 : int(len(good_posture) * .8)]
test_good = good_posture[int(len(good_posture) * .8) : len(good_posture)]

random.shuffle(bad_posture)
train_bad = bad_posture[0 : int(len(bad_posture) * .8)]
test_bad = bad_posture[int(len(bad_posture) * .8) : len(bad_posture)]

training_data = []
class_labels = np.array([])

for sample in train_good:
    training_data.append(extract_signal_features(sample))
    class_labels = np.append(class_labels, "good")
    
for sample in train_bad:
    training_data.append(extract_signal_features(sample))
    class_labels = np.append(class_labels, "bad")

clf = svm.SVC(kernel='linear', C=1, gamma=.5)
clf.fit(np.array(training_data), class_labels)

# evaluate model on test data
correct = 0
total = 0

for sample in test_good:
    svmPrediction = clf.predict(np.array(extract_signal_features(sample)).reshape(1, -1))
    if svmPrediction[0] == "good":
        correct += 1
    total += 1

goodCorrect = correct
goodTotal = total
print("Correct Good Posture Estimations: " + str(correct) + " out of " + str(total))
    
for sample in test_bad:
    svmPrediction = clf.predict(np.array(extract_signal_features(sample)).reshape(1, -1))
    if svmPrediction[0] == "bad":
        correct += 1
    total += 1

print("Correct Bad Posture Estimations: " + str(correct - goodCorrect) + " out of " + str(total - goodTotal))
print("Total Correct Estimations: " + str(correct) + " out of " + str(total) + " Acc: " + str(correct/total))

Correct Good Posture Estimations: 204 out of 208
Correct Bad Posture Estimations: 218 out of 227
Total Correct Estimations: 422 out of 435 Acc: 0.9701149425287356


In [5]:
# uses sklearn_porter to convert trained model to Java class
# https://github.com/nok/sklearn-porter
from sklearn_porter import Porter

porter = Porter(clf, language='java')
output = porter.export()

print(output)

class SVC {

    private enum Kernel { LINEAR, POLY, RBF, SIGMOID }

    private int nClasses;
    private int nRows;
    private int[] classes;
    private double[][] vectors;
    private double[][] coefficients;
    private double[] intercepts;
    private int[] weights;
    private Kernel kernel;
    private double gamma;
    private double coef0;
    private double degree;

    public SVC (int nClasses, int nRows, double[][] vectors, double[][] coefficients, double[] intercepts, int[] weights, String kernel, double gamma, double coef0, double degree) {
        this.nClasses = nClasses;
        this.classes = new int[nClasses];
        for (int i = 0; i < nClasses; i++) {
            this.classes[i] = i;
        }
        this.nRows = nRows;

        this.vectors = vectors;
        this.coefficients = coefficients;
        this.intercepts = intercepts;
        this.weights = weights;

        this.kernel = Kernel.valueOf(kernel.toUpperCase());
        this.gamma = gamma;
        thi

In [6]:
# helper functions for extracting features from segments  of sensor data, data_calc is a lambda function
def get_segment_mean(s_data, data_calc):
    avg = 0
    for s_g in s_data:
        avg += data_calc(s_g)
        
    return avg / len(s_data)

def get_segment_max(s_data, data_calc):
    s_max = data_calc(s_data[0])
    for s_g in s_data:
        if data_calc(s_g) > s_max:
            s_max = data_calc(s_g)
    return s_max

def get_segment_min(s_data, data_calc):
    s_min = data_calc(s_data[0])
    for s_g in s_data:
        if data_calc(s_g) < s_min:
            s_min = data_calc(s_g)
    return s_min

# extract features from segment of sensor data points
def extract_segment_features(s_data):
    features = []
    
    features.append(get_segment_max(s_data, lambda s_g: s_g.x1) - get_segment_min(s_data, lambda s_g: s_g.x1))
    features.append(get_segment_max(s_data, lambda s_g: s_g.y1) - get_segment_min(s_data, lambda s_g: s_g.y1))
    features.append(get_segment_max(s_data, lambda s_g: s_g.z1) - get_segment_min(s_data, lambda s_g: s_g.z1))
    
    features.append(get_segment_max(s_data, lambda s_g: s_g.x2) - get_segment_min(s_data, lambda s_g: s_g.x2))
    features.append(get_segment_max(s_data, lambda s_g: s_g.y2) - get_segment_min(s_data, lambda s_g: s_g.y2))
    features.append(get_segment_max(s_data, lambda s_g: s_g.z2) - get_segment_min(s_data, lambda s_g: s_g.z2))
    
    return features

In [11]:
# train and test walking model
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
import itertools
import time
import random
import numpy as np
import scipy as sp
from scipy import stats, signal

#randomize data sets and 80-20 train-test split them
#https://stackoverflow.com/questions/13423759/percent-list-slicing
random.shuffle(walking_segments)
train_walking = walking_segments[0 : int(len(walking_segments) * .8)]
test_walking = walking_segments[int(len(walking_segments) * .8) : len(walking_segments)]

random.shuffle(not_walking_segments)
train_not_walking = not_walking_segments[0 : int(len(not_walking_segments) * .8)]
test_not_walking = not_walking_segments[int(len(not_walking_segments) * .8) : len(not_walking_segments)]

training_data = []
class_labels = np.array([])

for sample in train_walking:
    training_data.append(extract_segment_features(sample))
    class_labels = np.append(class_labels, "walking")
    
for sample in train_not_walking:
    training_data.append(extract_segment_features(sample))
    class_labels = np.append(class_labels, "not walking")
    
clf = svm.SVC(kernel='linear', C=1, gamma=.5)
clf.fit(np.array(training_data), class_labels)
    
# evaluate model on test data
correct = 0
total = 0

for sample in test_walking:
    svmPrediction = clf.predict(np.array(extract_segment_features(sample)).reshape(1, -1))
    if svmPrediction[0] == "walking":
        correct += 1
    total += 1

walkingCorrect = correct
walkingTotal = total
print("Correct Walking Predictions: " + str(correct) + " out of " + str(total))
    
for sample in test_not_walking:
    svmPrediction = clf.predict(np.array(extract_segment_features(sample)).reshape(1, -1))
    if svmPrediction[0] == "not walking":
        correct += 1
    total += 1

print("Correct Not Walking Predictions: " + str(correct - walkingCorrect) + " out of " + str(total - walkingTotal))
print("Total Correct Predictions: " + str(correct) + " out of " + str(total) + " Acc: " + str(correct/total))

Correct Walking Predictions: 7 out of 7
Correct Not Walking Predictions: 17 out of 17
Total Correct Predictions: 24 out of 24 Acc: 1.0


In [12]:
# uses sklearn_porter to convert trained model to Java class
# https://github.com/nok/sklearn-porter
from sklearn_porter import Porter

porter = Porter(clf, language='java')
output = porter.export()

print(output)


class SVC {

    private enum Kernel { LINEAR, POLY, RBF, SIGMOID }

    private int nClasses;
    private int nRows;
    private int[] classes;
    private double[][] vectors;
    private double[][] coefficients;
    private double[] intercepts;
    private int[] weights;
    private Kernel kernel;
    private double gamma;
    private double coef0;
    private double degree;

    public SVC (int nClasses, int nRows, double[][] vectors, double[][] coefficients, double[] intercepts, int[] weights, String kernel, double gamma, double coef0, double degree) {
        this.nClasses = nClasses;
        this.classes = new int[nClasses];
        for (int i = 0; i < nClasses; i++) {
            this.classes[i] = i;
        }
        this.nRows = nRows;

        this.vectors = vectors;
        this.coefficients = coefficients;
        this.intercepts = intercepts;
        this.weights = weights;

        this.kernel = Kernel.valueOf(kernel.toUpperCase());
        this.gamma = gamma;
        thi